In [1]:
import pandas as pd
import geopandas as gpd
pd.set_option('mode.chained_assignment','raise')

/home/spierenburg/Documents/PhD/Study_1/code/study_1_venv/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import numpy as np
from scipy.stats import hypergeom,nhypergeom
from scipy.special import binom
from decimal import Decimal,ROUND_HALF_UP

In [3]:
def calc_Pa(L,S):
    '''
    This function computes the probability that the sum of L (L<=5) dices with 5 sides (-2,-1,0,1,2) 
    and 5-L dices with 3 sides (0,1,2) is S:
           Parameters:
                    S (int): Number.
                    L (int): Number of dices with 5 sides among the total number of dices.

            Returns:
                    P (float): Probability that the sum of the dices is S.
    
    See documentation for more detail.
                    
    '''
    if L>5 or L<0:
        raise Exception('L out of bound.')
    # Cases where the sum is lower or larger than the min/max alternative possible.
    if (S < -2 * L or S > 10):
        return 0
    
    P = 0
    for k in range(L+1):
        P_a = 0
        for i in range(5-L+1):
            if (S + 2*L - 5*k - i*3 + 4) <= 0:
                continue
            P_a = P_a + (-1)**i * binom(5-L,i) * binom(S + 2*L - 5*k - i*3 + 4,4)
        P_b = (-1)**k * binom(L,k) 
        P = P + P_a * P_b
    
    U = (5**L)*(3**(5-L))
    P = P/U
    return P

In [4]:
def calc_Pg(L,S):
    '''
    This function computes the probability that the sum of L (L<=2) dices with 5 sides (-2,1,0,1,2) 
    and 2-L dices with 3 sides (0,1,2) is S:
           Parameters:
                    S (int): Number.
                    L (int): Number of dices with 5 sides among 5 dices.

            Returns:
                    P (float): Probability that the sum of the dices is S.
    
    See documentation for more detail.
                    
    '''
    if L>2 or L<0:
        raise Exception('L out of bound.')  
    # Cases where the sum is lower or larger than the min/max alternative possible.
    if (S < -2 * L or S > 4):
        return 0
    
    P = 0
    for k in range(L+1):
        P_a = 0
        for i in range(2-L+1):
            if (S + 2*L - 5*k - i*3 + 1) <= 0:
                continue
            P_a = P_a + (-1)**i * binom(2-L,i) * (S + 2*L - 5*k - i*3 + 1)
        P_b = (-1)**k * binom(L,k) 
        P = P + P_a * P_b
    
    U = (5**L)*(3**(2-L))
    P = P/U
    return P

In [5]:
def calc_Phh(L,S):
    '''
    This function computes the probability that the sum of L (L<=4) dices with 5 sides (-2,-1,0,1,2) 
    and 4-L dices with 3 sides (0,1,2) is S:
           Parameters:
                    S (int): Number.
                    L (int): Number of dices with 5 sides among the total number of dices.

            Returns:
                    P (float): Probability that the sum of the dices is S.
    
    See documentation for more detail.
                    
    '''
    
    if L>4 or L<0:
        raise Exception('L out of bound.')
    # Cases where the sum is lower or larger than the min/max alternative possible.
    if (S < -2 * L or S > 8):
        return 0
    
    P = 0
    for k in range(L+1):
        P_a = 0
        for i in range(4-L+1):
            if (S + 2*L - 5*k - i*3 + 3) <= 0:
                continue
            P_a = P_a + (-1)**i * binom(4-L,i) * binom(S + 2*L - 5*k - i*3 + 3,3)
        P_b = (-1)**k * binom(L,k) 
        P = P + P_a * P_b
    
    U = (5**L)*(3**(4-L))
    P = P/U
    return P

In [6]:
def proba_age_pop(zones, id_col_p,id_col_014,
                  id_col_1524,id_col_2544,
                  id_col_4564,id_col_65PL):
    '''
    This function determines the probability that the pop equals (looking at the pop per age group):
        -the rounded pop - 2
        -the rounded pop - 1
        -the rounded pop
        -the rounded pop + 1
        -the rounded pop + 2.
        
            Parameters:
                    zones (DataFrame): Dataset containing the variables of interest.
                    id_col_p (str): Name of population column.
                    id_col_014 (str): Name of population column for age group 0-14.
                    id_col_1524 (str): Name of population column for age group 15-24.
                    id_col_2544 (str): Name of population column for age group 25-44.
                    id_col_4564 (str): Name of population column for age group 45-64.
                    id_col_65PL (str): Name of population column for age group >= 65.

            Returns:
                    zones (DataFrame): Dataset with 5 columns corresponding to the probability that the population equals:
                        -the rounded population - 2
                        -the rounded population - 1
                        -the rounded population
                        -the rounded population + 1
                        -the rounded population + 2.
                        
    Further description:
    The true population in each age group in each zone has 5 possible values: 
        -the rounded pop in age group x - 2
        -the rounded pop - 1
        -the rounded pop
        -the rounded pop + 1
        -the rounded pop + 2.
        
    It is modelled using a throw of 5 dices with 5 sides: -2,-1,0,1,2
        
    Exception: When the rounded population is 5, the actual population is 5,6 or 7.
    '''
    zones = zones.copy()
    # ldiff for lowest possible difference between the actual value and the rounded value
    # when rounding to the closest 5.
    
    zones['L'] = 5 - zones[[id_col_014,
                            id_col_1524,
                            id_col_2544,
                            id_col_4564,
                            id_col_65PL]].eq(5).sum(1)

    # Sum of population in all age group.
    zones['S_a'] = zones.loc[:,[id_col_014,id_col_1524,id_col_2544,id_col_4564,id_col_65PL]].sum(axis = 1)
    zones['S_n2'] = zones[id_col_p] - zones['S_a'] - 2
    zones['S_n1'] = zones[id_col_p] - zones['S_a'] - 1
    zones['S_0']  = zones[id_col_p] - zones['S_a']
    zones['S_p1'] = zones[id_col_p] - zones['S_a'] + 1
    zones['S_p2'] = zones[id_col_p] - zones['S_a'] + 2

    zones['Pa_n2'] = zones.apply(lambda x: calc_Pa(x.L, x.S_n2), axis = 1)
    zones['Pa_n1'] = zones.apply(lambda x: calc_Pa(x.L, x.S_n1), axis = 1)
    zones['Pa_0']  = zones.apply(lambda x: calc_Pa(x.L, x.S_0), axis = 1)
    zones['Pa_p1'] = zones.apply(lambda x: calc_Pa(x.L, x.S_p1), axis = 1)
    zones['Pa_p2'] = zones.apply(lambda x: calc_Pa(x.L, x.S_p2), axis = 1)
    
    
    zones['sum_P'] = zones.loc[:,['Pa_n2','Pa_n1','Pa_0','Pa_p1','Pa_p2']].sum(axis = 1)
    zones['Pa_n2'] =  zones.loc[:,'Pa_n2']/zones.loc[:,'sum_P']
    zones['Pa_n1'] =  zones.loc[:,'Pa_n1']/zones.loc[:,'sum_P']
    zones['Pa_0']  =  zones.loc[:,'Pa_0']/zones.loc[:,'sum_P']
    zones['Pa_p1'] =  zones.loc[:,'Pa_p1']/zones.loc[:,'sum_P']
    zones['Pa_p2'] =  zones.loc[:,'Pa_p2']/zones.loc[:,'sum_P']
    zones['Pa_n2'] =  zones['Pa_n2'].mask(zones[id_col_p] == 0, 0)
    zones['Pa_n1'] =  zones['Pa_n1'].mask(zones[id_col_p] == 0, 0)
    zones['Pa_0']  =  zones['Pa_0'].mask(zones[id_col_p] == 0, 1)
    zones['Pa_p1'] =  zones['Pa_p1'].mask(zones[id_col_p] == 0, 0)
    zones['Pa_p2'] =  zones['Pa_p2'].mask(zones[id_col_p] == 0, 0)
    
    zones = zones.drop(columns = ['S_a','S_n2','S_n1','S_0','S_p1','S_p2','sum_P','L'])
    return zones

In [7]:
def proba_gender_pop(zones, id_col_p, id_col_M, id_col_W):
    '''
    This function determines the probability that the pop equals (looking at the pop per gender):
        -the rounded pop - 2
        -the rounded pop - 1
        -the rounded pop
        -the rounded pop + 1
        -the rounded pop + 2.
        
            Parameters:
                    zones (DataFrame): Dataset containing the variables of interest.
                    id_col_p (str): Name of population column.
                    id_col_M (str): Name of column containing number of men.
                    id_col_W (str): Name of column containing number of women.
                    
            Returns:
                    zones (DataFrame): Dataset with 5 columns corresponding to the probability that the population equals:
                        -the rounded population - 2
                        -the rounded population - 1
                        -the rounded population
                        -the rounded population + 1
                        -the rounded population + 2.
                        
    Further description:

    
    The true population in each zone has 5 possible values: 
        -the rounded pop - 2
        -the rounded pop - 1
        -the rounded pop
        -the rounded pop + 1
        -the rounded pop + 2.
        
        
    It is modelled using a throw of 2 dices with 5 sides: -2,-1,0,1,2
    '''
    
    zones = zones.copy()

    # N_min is the lowest pop obtained by summing the pop from the two genders.

    zones['L'] = 2 - zones[[id_col_M,
                            id_col_W]].eq(5).sum(1)


    zones['S_g'] = zones.loc[:,[id_col_M, id_col_W]].sum(axis = 1)
    # Sum of 5 dices (with 5 sides: -2,-1,0,1,2).
    zones['S_n2'] = zones[id_col_p] - zones['S_g'] - 2
    zones['S_n1'] = zones[id_col_p] - zones['S_g'] - 1
    zones['S_0']  = zones[id_col_p] - zones['S_g']
    zones['S_p1'] = zones[id_col_p] - zones['S_g'] + 1
    zones['S_p2'] = zones[id_col_p] - zones['S_g'] + 2

    zones['Pg_n2'] = zones.apply(lambda x: calc_Pg(x.L, x.S_n2), axis = 1)
    zones['Pg_n1'] = zones.apply(lambda x: calc_Pg(x.L, x.S_n1), axis = 1)
    zones['Pg_0']  = zones.apply(lambda x: calc_Pg(x.L, x.S_0), axis = 1)
    zones['Pg_p1'] = zones.apply(lambda x: calc_Pg(x.L, x.S_p1), axis = 1)
    zones['Pg_p2'] = zones.apply(lambda x: calc_Pg(x.L, x.S_p2), axis = 1)

    zones['sum_P'] = zones.loc[:,['Pg_n2','Pg_n1','Pg_0','Pg_p1','Pg_p2']].sum(axis = 1)
    zones['Pg_n2'] =  zones.loc[:,'Pg_n2']/zones.loc[:,'sum_P']
    zones['Pg_n1'] =  zones.loc[:,'Pg_n1']/zones.loc[:,'sum_P']
    zones['Pg_0']  =  zones.loc[:,'Pg_0'] /zones.loc[:,'sum_P']
    zones['Pg_p1'] =  zones.loc[:,'Pg_p1']/zones.loc[:,'sum_P']
    zones['Pg_p2'] =  zones.loc[:,'Pg_p2']/zones.loc[:,'sum_P']
    
    zones['Pg_n2'] =  zones['Pg_n2'].mask(zones[id_col_p] == 0, 0)
    zones['Pg_n1'] =  zones['Pg_n1'].mask(zones[id_col_p] == 0, 0)
    zones['Pg_0']  =  zones['Pg_0'].mask(zones[id_col_p] == 0, 1)
    zones['Pg_p1'] =  zones['Pg_p1'].mask(zones[id_col_p] == 0, 0)
    zones['Pg_p2'] =  zones['Pg_p2'].mask(zones[id_col_p] == 0, 0)

    zones = zones.drop(columns = ['S_g','S_n2','S_n1','S_0','S_p1','S_p2','sum_P','L'])
    
    return zones

In [8]:
def proba_hh(zones, id_col_Nhh,id_col_hh1, 
             id_col_hh2, id_col_hh3, id_col_hh4):
    '''
    This function determines the probability that the number of households equals 
    (looking at the household per group category):
        -the rounded number of households - 2
        -the rounded number of households - 1
        -the rounded number of households
        -the rounded number of households + 1
        -the rounded number of households + 2.
        
            Parameters:
                    zones (DataFrame): Dataset containing the variables of interest.
                    id_col_Nhh (str): Name of the household column.
                    id_col_hh1 (str): Name of the category 1 household column. 
                    id_col_hh2 (str): Name of the category 2 household column.
                    id_col_hh3 (str): Name of the category 3 household column.
                    id_col_hh4 (str): Name of the category 4 household column.
                    
            Returns:
                    zones (DataFrame): Dataset with 5 columns corresponding to the probability that the population equals:
                        -the rounded number of households - 2
                        -the rounded number of households - 1
                        -the rounded number of households
                        -the rounded number of households + 1
                        -the rounded number of households + 2.
                        
    Further description:

    
    The true number of households in each zone has 5 possible values: 
        -the rounded number of households - 2
        -the rounded number of households - 1
        -the rounded number of households
        -the rounded number of households + 1
        -the rounded number of households + 2.
        
        
    It is modelled using a throw of 4 dices with 5 sides: -2,-1,0,1,2
    '''
    
    zones = zones.copy()

    # N_min is the lowest pop obtained by summing the pop from the two genders.

    zones['L'] = 4 - zones[[id_col_hh1,
                            id_col_hh2,
                            id_col_hh3, 
                            id_col_hh4]].eq(5).sum(1)


    zones['S_hh'] = zones.loc[:,[id_col_hh1,id_col_hh2,
                                 id_col_hh3, id_col_hh4]].sum(axis = 1)
    # Sum of 5 dices (with 5 sides: -2,-1,0,1,2).
    zones['S_n2'] = zones[id_col_Nhh] - zones['S_hh'] - 2
    zones['S_n1'] = zones[id_col_Nhh] - zones['S_hh'] - 1
    zones['S_0']  = zones[id_col_Nhh] - zones['S_hh']
    zones['S_p1'] = zones[id_col_Nhh] - zones['S_hh'] + 1
    zones['S_p2'] = zones[id_col_Nhh] - zones['S_hh'] + 2

    zones['Phh_n2'] = zones.apply(lambda x: calc_Phh(x.L, x.S_n2), axis = 1)
    zones['Phh_n1'] = zones.apply(lambda x: calc_Phh(x.L, x.S_n1), axis = 1)
    zones['Phh_0']  = zones.apply(lambda x: calc_Phh(x.L, x.S_0), axis = 1)
    zones['Phh_p1'] = zones.apply(lambda x: calc_Phh(x.L, x.S_p1), axis = 1)
    zones['Phh_p2'] = zones.apply(lambda x: calc_Phh(x.L, x.S_p2), axis = 1)

    zones['sum_P'] = zones.loc[:,['Phh_n2','Phh_n1',
                                  'Phh_0','Phh_p1','Phh_p2']].sum(axis = 1)
    
    
    zones['Phh_n2'] = zones['Phh_n2'].mask(zones['sum_P'] == 0,1)
    zones['Phh_n1'] = zones['Phh_n1'].mask(zones['sum_P'] == 0,1)
    zones['Phh_0'] = zones['Phh_0'].mask(zones['sum_P'] == 0,1)
    zones['Phh_p1'] = zones['Phh_p1'].mask(zones['sum_P'] == 0,1)
    zones['Phh_p2'] = zones['Phh_p2'].mask(zones['sum_P'] == 0,1)
    zones['sum_P'] = zones['sum_P'].mask(zones['sum_P'] == 0,5)
    
    zones['Phh_n2'] =  zones.loc[:,'Phh_n2']/zones.loc[:,'sum_P']
    zones['Phh_n1'] =  zones.loc[:,'Phh_n1']/zones.loc[:,'sum_P']
    zones['Phh_0']  =  zones.loc[:,'Phh_0'] /zones.loc[:,'sum_P']
    zones['Phh_p1'] =  zones.loc[:,'Phh_p1']/zones.loc[:,'sum_P']
    zones['Phh_p2'] =  zones.loc[:,'Phh_p2']/zones.loc[:,'sum_P']

    zones = zones.drop(columns = ['S_hh','S_n2','S_n1','S_0','S_p1','S_p2','sum_P','L'])
    
    return zones

In [9]:
def proba_hh_pop(zones, id_col_p, id_col_Nhh,id_col_pop_per_hh,
                 Phh_n2,Phh_n1,Phh_0,Phh_p1,Phh_p2):
    '''
    This function determines the probability that the population equals 
    (looking at the number of households in the zone):
        -the rounded pop - 2
        -the rounded pop - 1
        -the rounded pop
        -the rounded pop + 1
        -the rounded pop + 2.
        
            Parameters:
                    zones (DataFrame): Dataset containing the variables of interest.
                    id_col_p (str): Name of population column.
                    id_col_Nhh (str): Name of the household column.
                    id_col_pop_per_hh (str): Name of column containing average pop per household.
                    Phh_n2 (str): Name of the column containing the probability that the number of households is the rounded number of households -2.
                    Phh_n1 (str): Name of the column containing the probability that the number of households is the rounded number of households -1.
                    Phh_0 (str): Name of the column containing the probability that the number of households is the rounded number of households.
                    Phh_p1 (str): Name of the column containing the probability that the number of households is the rounded number of households +1.
                    Phh_p2 (str): Name of the column containing the probability that the number of households is the rounded number of households +2.
                    
            Returns:
                    zones (DataFrame): Dataset with 5 columns corresponding to the probability that the population equals:
                        -the rounded population - 2
                        -the rounded population - 1
                        -the rounded population
                        -the rounded population + 1
                        -the rounded population + 2.
                        
    Further description:

    
    '''
    
    zones = zones.copy()

    # First pop second hh.
    zones = zones.assign(Pn2n2 = 1,Pn2n1 = 1,Pn20 = 1,Pn2p1 = 1,Pn2p2 = 1,
                         Pn1n2 = 1,Pn1n1 = 1,Pn10 = 1,Pn1p1 = 1,Pn1p2 = 1,
                         P0n2 = 1,P0n1 = 1,P00 = 1,P0p1 = 1,P0p2 = 1,
                         Pp1n2 = 1,Pp1n1 = 1,Pp10 = 1,Pp1p1 = 1,Pp1p2 = 1,
                         Pp2n2 = 1,Pp2n1 = 1,Pp20 = 1,Pp2p1 = 1,Pp2p2 = 1)

    zones = zones.assign(rn2n2 = (zones[id_col_p] - 2).mask(zones[id_col_p] <= 2, 0) / \
                                 (zones[id_col_Nhh] - 2).mask(zones[id_col_Nhh] <= 2, 1),
                         rn2n1 = (zones[id_col_p] - 2).mask(zones[id_col_p] <= 2, 0) / \
                                 (zones[id_col_Nhh] - 1).mask(zones[id_col_Nhh] <= 1, 1),
                         rn20 = (zones[id_col_p] - 2).mask(zones[id_col_p] <= 2, 0) / \
                                (zones[id_col_Nhh]).mask(zones[id_col_Nhh] <= 0, 1),
                         rn2p1 = (zones[id_col_p] - 2).mask(zones[id_col_p] <= 2, 0) / \
                                 (zones[id_col_Nhh] + 1),
                         rn2p2 = (zones[id_col_p] - 2).mask(zones[id_col_p] <= 2, 0) / \
                                 (zones[id_col_Nhh] + 2),
                         rn1n2 = (zones[id_col_p] - 1).mask(zones[id_col_p] <= 1, 0) / \
                                 (zones[id_col_Nhh] - 2).mask(zones[id_col_Nhh] <= 2, 1),
                         rn1n1 = (zones[id_col_p] - 1).mask(zones[id_col_p] <= 1, 0) / \
                                 (zones[id_col_Nhh] - 1).mask(zones[id_col_Nhh] <= 1, 1),
                         rn10 = (zones[id_col_p] - 1).mask(zones[id_col_p] <= 1, 0) / \
                                 (zones[id_col_Nhh]).mask(zones[id_col_Nhh] <= 0, 1),
                         rn1p1 = (zones[id_col_p] - 1).mask(zones[id_col_p] <= 1, 0) / \
                                 (zones[id_col_Nhh] + 1),
                         rn1p2 = (zones[id_col_p] - 1).mask(zones[id_col_p] <= 1, 0) / \
                                 (zones[id_col_Nhh] + 2),
                         r0n2 = (zones[id_col_p]) / (zones[id_col_Nhh] - 2).mask(zones[id_col_Nhh] <= 2, 1),
                         r0n1 = (zones[id_col_p]) / (zones[id_col_Nhh] - 1).mask(zones[id_col_Nhh] <= 1, 1),
                         r00 = (zones[id_col_p]) / (zones[id_col_Nhh]).mask(zones[id_col_Nhh] <= 0, 1),
                         r0p1 = (zones[id_col_p]) / (zones[id_col_Nhh] + 1),
                         r0p2 = (zones[id_col_p]) / (zones[id_col_Nhh] + 2),
                         rp1n2 = (zones[id_col_p] + 1) / (zones[id_col_Nhh] - 2).mask(zones[id_col_Nhh] <= 2, 1),
                         rp1n1 = (zones[id_col_p] + 1) / (zones[id_col_Nhh] - 1).mask(zones[id_col_Nhh] <= 1, 1),
                         rp10 = (zones[id_col_p]  + 1) / (zones[id_col_Nhh]).mask(zones[id_col_Nhh] <= 0, 1),
                         rp1p1 = (zones[id_col_p] + 1) / (zones[id_col_Nhh] + 1),
                         rp1p2 = (zones[id_col_p] + 1) / (zones[id_col_Nhh] + 2),
                         rp2n2 = (zones[id_col_p] + 2) / (zones[id_col_Nhh] - 2).mask(zones[id_col_Nhh] <= 2, 1),
                         rp2n1 = (zones[id_col_p] + 2) / (zones[id_col_Nhh] - 1).mask(zones[id_col_Nhh] <= 1, 1),
                         rp20 = (zones[id_col_p]  + 2) / (zones[id_col_Nhh]).mask(zones[id_col_Nhh] <= 0, 1),
                         rp2p1 = (zones[id_col_p] + 2) / (zones[id_col_Nhh] + 1),
                         rp2p2 = (zones[id_col_p] + 2) / (zones[id_col_Nhh] + 2))
    
    
    
    zones.loc[:,['rn2n2','rn2n1','rn20','rn2p1','rn2p2',
                 'rn1n2','rn1n1','rn10','rn1p1','rn1p2',
                 'r0n2','r0n1','r00','r0p1','r0p2',
                 'rp1n2','rp1n1','rp10','rp1p1','rp1p2',
                 'rp2n2','rp2n1','rp20','rp2p1','rp2p2']] = \
    zones.loc[:,['rn2n2','rn2n1','rn20','rn2p1','rn2p2',
                 'rn1n2','rn1n1','rn10','rn1p1','rn1p2',
                 'r0n2','r0n1','r00','r0p1','r0p2',
                 'rp1n2','rp1n1','rp10','rp1p1','rp1p2',
                 'rp2n2','rp2n1','rp20','rp2p1','rp2p2']].applymap(lambda x: Decimal(str(x)).quantize(Decimal('.1'), 
                                                                                                      rounding=ROUND_HALF_UP)).astype(float)
    
    zones.loc[:,['Pn2n2','Pn2n1','Pn20','Pn2p1','Pn2p2',
                 'Pn1n2','Pn1n1','Pn10','Pn1p1','Pn1p2',
                 'P0n2','P0n1','P00','P0p1','P0p2',
                 'Pp1n2','Pp1n1','Pp10','Pp1p1','Pp1p2',
                 'Pp2n2','Pp2n1','Pp20','Pp2p1','Pp2p2']] = \
    (zones.loc[:,['rn2n2','rn2n1','rn20','rn2p1','rn2p2',
                 'rn1n2','rn1n1','rn10','rn1p1','rn1p2',
                 'r0n2','r0n1','r00','r0p1','r0p2',
                 'rp1n2','rp1n1','rp10','rp1p1','rp1p2',
                 'rp2n2','rp2n1','rp20','rp2p1','rp2p2']].values == 
     zones[[id_col_pop_per_hh]].values).astype(int)
        
    zones.loc[:,['Pn2n2','Pn1n2',
             'P0n2','Pp1n2','Pp2n2']] = zones[['Pn2n2','Pn1n2',
                                               'P0n2','Pp1n2','Pp2n2']].multiply(zones['Phh_n2'],
                                                                                 axis = 0)
    zones.loc[:,['Pn2n1','Pn1n1',
                 'P0n1','Pp1n1','Pp2n1']] = zones[['Pn2n1','Pn1n1',
                                                   'P0n1','Pp1n1','Pp2n1']].multiply(zones['Phh_n1'],
                                                                                     axis = 0)
    zones.loc[:,['Pn20','Pn10',
                 'P00','Pp10','Pp20']] = zones[['Pn20','Pn10',
                                                'P00','Pp10','Pp20']].multiply(zones['Phh_0'],
                                                                               axis = 0)
    zones.loc[:,['Pn2p1','Pn1p1',
                 'P0p1','Pp1p1','Pp2p1']] = zones[['Pn2p1','Pn1p1',
                                                   'P0p1','Pp1p1','Pp2p1']].multiply(zones['Phh_p1'],
                                                                                     axis = 0)
    zones.loc[:,['Pn2p2','Pn1p2',
                 'P0p2','Pp1p2','Pp2p2']] = zones[['Pn2p2','Pn1p2',
                                                   'P0p2','Pp1p2','Pp2p2']].multiply(zones['Phh_p2'],
                                                                                     axis = 0)
    
    zones['Ph_n2'] = zones[['Pn2n2','Pn2n1',
                            'Pn20','Pn2p1','Pn2p2']].sum(axis = 1)
    zones['Ph_n1'] = zones[['Pn1n2','Pn1n1',
                            'Pn10','Pn1p1','Pn1p2']].sum(axis = 1)
    zones['Ph_0'] = zones[['P0n2','P0n1',
                           'P00','P0p1','P0p2']].sum(axis = 1)
    zones['Ph_p1'] = zones[['Pp1n2','Pp1n1',
                            'Pp10','Pp1p1','Pp1p2']].sum(axis = 1)
    zones['Ph_p2'] = zones[['Pp2n2','Pp2n1',
                            'Pp20','Pp2p1','Pp2p2']].sum(axis = 1)
    
    zones['Ph_n2'] = zones['Ph_n2'].mask(zones[id_col_Nhh] == 0,1)
    zones['Ph_n1'] = zones['Ph_n1'].mask(zones[id_col_Nhh] == 0,1)
    zones['Ph_0'] = zones['Ph_0'].mask(zones[id_col_Nhh] == 0,1)
    zones['Ph_p1'] = zones['Ph_p1'].mask(zones[id_col_Nhh] == 0,1)
    zones['Ph_p2'] = zones['Ph_p2'].mask(zones[id_col_Nhh] == 0,1)
    
    zones['sum_P'] = zones.loc[:,['Ph_n2','Ph_n1',
                                  'Ph_0','Ph_p1','Ph_p2']].sum(axis = 1)
        
    zones['Ph_n2'] = zones['Ph_n2'].mask(zones['sum_P'] == 0,1)
    zones['Ph_n1'] = zones['Ph_n1'].mask(zones['sum_P'] == 0,1)
    zones['Ph_0'] = zones['Ph_0'].mask(zones['sum_P'] == 0,1)
    zones['Ph_p1'] = zones['Ph_p1'].mask(zones['sum_P'] == 0,1)
    zones['Ph_p2'] = zones['Ph_p2'].mask(zones['sum_P'] == 0,1)
    zones['sum_P'] = zones['sum_P'].mask(zones['sum_P'] == 0,5)
    
    zones['Ph_n2'] = zones.loc[:,'Ph_n2']/zones.loc[:,'sum_P']
    zones['Ph_n1'] = zones.loc[:,'Ph_n1']/zones.loc[:,'sum_P']
    zones['Ph_0']  = zones.loc[:,'Ph_0'] /zones.loc[:,'sum_P']
    zones['Ph_p1'] = zones.loc[:,'Ph_p1']/zones.loc[:,'sum_P']
    zones['Ph_p2'] = zones.loc[:,'Ph_p2']/zones.loc[:,'sum_P']
    
   

    zones = zones.drop(columns = ['rn2n2','rn2n1','rn20','rn2p1','rn2p2',
                                  'rn1n2','rn1n1','rn10','rn1p1','rn1p2',
                                  'r0n2','r0n1','r00','r0p1','r0p2',
                                  'rp1n2','rp1n1','rp10','rp1p1','rp1p2',
                                  'rp2n2','rp2n1','rp20','rp2p1','rp2p2',
                                  'Pn2n2','Pn2n1','Pn20','Pn2p1','Pn2p2',
                                  'Pn1n2','Pn1n1','Pn10','Pn1p1','Pn1p2',
                                  'P0n2','P0n1','P00','P0p1','P0p2',
                                  'Pp1n2','Pp1n1','Pp10','Pp1p1','Pp1p2',
                                  'Pp2n2','Pp2n1','Pp20','Pp2p1','Pp2p2','sum_P'])
    
    return zones

In [10]:
PC6 = gpd.read_file('../data/raw_data/sociodemographics/PC6/CBS_PC6_2017_v3.shp')

PC6_won = PC6.loc[:,['Postcode','WONING']].copy()

PC6 = PC6.loc[:,['Postcode','INWONER','P_NL_ACHTG','P_WE_MIG_A','P_NW_MIG_A','MAN',
                 'VROUW','INW_014','INW_1524','INW_2544','INW_4564','INW_65PL','AANTAL_HH',
                 'TOTHH_EENP', 'TOTHH_MPZK','HH_EENOUD','HH_TWEEOUD','GEM_HH_GR','geometry']]

PC6 = PC6.rename(columns = {'INWONER':'pop',
                            'MAN':'pop_M',
                            'VROUW':'pop_V',
                            'INW_014':'pop_014',
                            'INW_1524':'pop_1524',
                            'INW_2544':'pop_2544',
                            'INW_4564':'pop_4564',
                            'INW_65PL':'pop_65PL',
                            'AANTAL_HH':'N_HH',
                            'TOTHH_EENP':'HH1',
                            'TOTHH_MPZK':'HH2',
                            'HH_EENOUD':'HH3',
                            'HH_TWEEOUD':'HH4',
                            'GEM_HH_GR':'pop_per_HH'})

PC6['N_HH_ori'] = PC6['N_HH'].copy()
PC6['pop_per_HH_ori'] = PC6['pop_per_HH'].copy()

PC6 = PC6.replace(-99997,0)

# If the population is 0 in the data set, it can be 0 to 4 in reality.
# We set it to 2 by default (the value is edited later on).
PC6.loc[:,'pop_014'] = PC6['pop_014'].mask((PC6['pop_014'] == 0) & (PC6['pop'] != 0), 2)
PC6.loc[:,'pop_1524'] = PC6['pop_1524'].mask((PC6['pop_1524'] == 0) &  (PC6['pop'] != 0), 2)
PC6.loc[:,'pop_2544'] = PC6['pop_2544'].mask((PC6['pop_2544'] == 0) &  (PC6['pop'] != 0), 2)
PC6.loc[:,'pop_4564'] = PC6['pop_4564'].mask((PC6['pop_4564'] == 0) &  (PC6['pop'] != 0), 2)
PC6.loc[:,'pop_65PL'] = PC6['pop_65PL'].mask((PC6['pop_65PL'] == 0) &  (PC6['pop'] != 0), 2)
PC6.loc[:,'pop_M'] = PC6['pop_M'].mask((PC6['pop_M'] == 0) &  (PC6['pop'] != 0), 2)
PC6.loc[:,'pop_V'] = PC6['pop_V'].mask((PC6['pop_V'] == 0) & (PC6['pop'] != 0), 2)
PC6.loc[:,'HH1'] = PC6['HH1'].mask((PC6['HH1'] == 0) & (PC6['N_HH'] != 0), 2)
PC6.loc[:,'HH2'] = PC6['HH2'].mask((PC6['HH2'] == 0) & (PC6['N_HH'] != 0), 2)
PC6.loc[:,'HH3'] = PC6['HH3'].mask((PC6['HH3'] == 0) & (PC6['N_HH'] != 0), 2)
PC6.loc[:,'HH4'] = PC6['HH4'].mask((PC6['HH4'] == 0) & (PC6['N_HH'] != 0), 2)

In [11]:
# Estimation of the actual number of inhabitants in each postcode.
PC6 = proba_age_pop(PC6,'pop','pop_014','pop_1524','pop_2544','pop_4564','pop_65PL')

PC6 = proba_gender_pop(PC6, 'pop','pop_M','pop_V')

PC6 = proba_hh(PC6,'N_HH','HH1','HH2','HH3','HH4') 

PC6 = proba_hh_pop(PC6, 'pop', 'N_HH','pop_per_HH','Phh_n2','Phh_n1','Phh_0','Phh_p1','Phh_p2')

PC6['Pn2'] = PC6['Pa_n2'] * PC6['Pg_n2'] * PC6['Ph_n2'] 
PC6['Pn1'] = PC6['Pa_n1'] * PC6['Pg_n1'] * PC6['Ph_n1'] 
PC6['P0']  = PC6['Pa_0']  * PC6['Pg_0']  * PC6['Ph_0'] 
PC6['Pp1'] = PC6['Pa_p1'] * PC6['Pg_p1'] * PC6['Ph_p1'] 
PC6['Pp2'] = PC6['Pa_p2'] * PC6['Pg_p2'] * PC6['Ph_p2'] 

PC6['P_tot'] = PC6[['Pn2','Pn1','P0','Pp1','Pp2']].sum(axis = 1)

PC6['Pn2'] = PC6['Pn2'].mask(PC6['P_tot'] == 0, PC6['Pa_n2'] * PC6['Pg_n2'])
PC6['Pn1'] = PC6['Pn1'].mask(PC6['P_tot'] == 0, PC6['Pa_n1'] * PC6['Pg_n1'])
PC6['P0']  = PC6['P0'].mask(PC6['P_tot'] == 0, PC6['Pa_0'] * PC6['Pg_0'])
PC6['Pp1'] = PC6['Pp1'].mask(PC6['P_tot'] == 0, PC6['Pa_p1'] * PC6['Pg_p1'])
PC6['Pp2'] = PC6['Pp2'].mask(PC6['P_tot'] == 0, PC6['Pa_p2'] * PC6['Pg_p2'])

PC6['P_tot'] = PC6['P_tot'].mask(PC6['P_tot'] == 0, PC6[['Pn2','Pn1','P0','Pp1','Pp2']].sum(axis = 1))

PC6['Pn2'] = PC6['Pn2'] / PC6['P_tot']
PC6['Pn1'] = PC6['Pn1'] / PC6['P_tot']
PC6['P0']  = PC6['P0']  / PC6['P_tot']
PC6['Pp1'] = PC6['Pp1'] / PC6['P_tot']
PC6['Pp2'] = PC6['Pp2'] / PC6['P_tot']

PC6['pop_est'] = (PC6['pop'] - 2) * PC6['Pn2'] + \
                 (PC6['pop'] - 1) * PC6['Pn1'] + \
                 (PC6['pop']    ) * PC6['P0']  + \
                 (PC6['pop'] + 1) * PC6['Pp1'] + \
                 (PC6['pop'] + 2) * PC6['Pp2']
PC6['pop_est'] = PC6['pop_est'].round().astype(int)

In [12]:
PC6 = PC6.loc[:,['Postcode','pop_est','P_NL_ACHTG', 'P_WE_MIG_A', 'P_NW_MIG_A','pop']]

In [13]:
# Computing the population per social group in each postcode.
# In the data, the population per social group is expressed in percentage terms.
# This value is rounded to the closest 10%. 
# If 54% of the population are individuals with a Western migration background, the value will be 50% in the data.
# We make a closer estimation below. For that, we compute the largest and lowest possible values for the population
# per group.

In [14]:
PC6['N_NL_min'] = (PC6['pop_est'] * (PC6['P_NL_ACHTG']-5) /100).round()
PC6['N_NL_min'] = PC6['N_NL_min'].mask(PC6['N_NL_min']<0,0)
# If the percentage is indicated, but there are at least 5 people from that group. 
# The percentage is not disclosed if there are less than 5 people from the group.
PC6['N_NL_min'] = PC6['N_NL_min'].mask((PC6['N_NL_min']<5) & (PC6['P_NL_ACHTG'] > 0),5)

PC6['N_NL_max'] = (PC6['pop_est'] * (PC6['P_NL_ACHTG']+5) /100).round()
# If the percentage is not disclosed, there are maximum 4 people from the group.
PC6['N_NL_max'] = PC6['N_NL_max'].mask((PC6['N_NL_max'] < 5) & (PC6['pop_est']>0),4)
PC6['N_NL_max'] = PC6['N_NL_max'].mask(PC6['N_NL_max']>PC6['pop_est'],PC6['pop_est'])

In [15]:
PC6['N_WE_min'] = (PC6['pop_est'] * (PC6['P_WE_MIG_A']-5) /100).round()
PC6['N_WE_min'] = PC6['N_WE_min'].mask(PC6['N_WE_min']<0,0)
PC6['N_WE_min'] = PC6['N_WE_min'].mask((PC6['N_WE_min']<5) &  (PC6['P_WE_MIG_A'] > 0),5)
PC6['N_WE_max'] = (PC6['pop_est'] * (PC6['P_WE_MIG_A']+5) /100).round()
PC6['N_WE_max'] = PC6['N_WE_max'].mask((PC6['N_WE_max'] < 5) &  (PC6['pop_est']>0),4)
PC6['N_WE_max'] = PC6['N_WE_max'].mask(PC6['N_WE_max']>PC6['pop_est'],PC6['pop_est'])

In [16]:
PC6['N_NW_min'] = (PC6['pop_est'] * (PC6['P_NW_MIG_A']-5) /100).round()
PC6['N_NW_min'] = PC6['N_NW_min'].mask(PC6['N_NW_min']<0,0)
PC6['N_NW_min'] = PC6['N_NW_min'].mask((PC6['N_NW_min']<5) &  (PC6['P_NW_MIG_A'] > 0),5)
PC6['N_NW_max'] = (PC6['pop_est'] * (PC6['P_NW_MIG_A']+5) /100).round()
PC6['N_NW_max'] = PC6['N_NW_max'].mask((PC6['N_NW_max'] < 5) &  (PC6['pop_est']>0),4)
PC6['N_NW_max'] = PC6['N_NW_max'].mask(PC6['N_NW_max']>PC6['pop_est'],PC6['pop_est'])

In [17]:
PC6['pop_est'] = PC6['pop_est'].mask(PC6['N_NL_min'] + PC6['N_WE_min'] + PC6['N_NW_min'] > PC6['pop_est'],
                                     PC6['N_NL_min'] + PC6['N_WE_min'] + PC6['N_NW_min'])

PC6['pop_est'] = PC6['pop_est'].mask((PC6['N_NL_max'] + PC6['N_WE_max'] + PC6['N_NW_max'] < PC6['pop_est']) & 
                                     (PC6['P_NL_ACHTG'] + PC6['P_WE_MIG_A'] + PC6['P_NW_MIG_A'] > 0),
                                     PC6['N_NL_max'] + PC6['N_WE_max'] + PC6['N_NW_max'])

PC6['N_NW_max'] = PC6['N_NW_max'].mask((PC6['N_NL_max'] + PC6['N_WE_max'] + PC6['N_NW_max'] < PC6['pop_est']) & 
                                       (PC6['P_NL_ACHTG'] + PC6['P_WE_MIG_A'] + PC6['P_NW_MIG_A'] == 0),
                                       PC6['pop_est'])
PC6['N_NL_max'] = PC6['N_NL_max'].mask((PC6['N_NL_max'] + PC6['N_WE_max'] + PC6['N_NW_max'] < PC6['pop_est']) & 
                                       (PC6['P_NL_ACHTG'] + PC6['P_WE_MIG_A'] + PC6['P_NW_MIG_A'] == 0),
                                       PC6['pop_est'])
PC6['N_WE_max'] = PC6['N_WE_max'].mask((PC6['N_NL_max'] + PC6['N_WE_max'] + PC6['N_NW_max'] < PC6['pop_est']) & 
                                       (PC6['P_NL_ACHTG'] + PC6['P_WE_MIG_A'] + PC6['P_NW_MIG_A'] == 0),
                                       PC6['pop_est'])

PC6['N_NL_max'] = PC6['N_NL_max'].mask(PC6['N_NL_min'] > PC6['N_NL_max'], PC6['N_NL_min'])
PC6['N_WE_max'] = PC6['N_WE_max'].mask(PC6['N_WE_min'] > PC6['N_WE_max'], PC6['N_WE_min'])
PC6['N_NW_max'] = PC6['N_NW_max'].mask(PC6['N_NW_min'] > PC6['N_NW_max'], PC6['N_NW_min'])

In [18]:
PC6['N_NL_min'] = PC6['N_NL_min'].mask((PC6['pop_est'] - PC6['N_WE_max'] - PC6['N_NW_max'] > PC6['N_NL_min']),
                                       PC6['pop_est'] - PC6['N_WE_max'] - PC6['N_NW_max'])
PC6['N_WE_min'] = PC6['N_WE_min'].mask((PC6['pop_est'] - PC6['N_NL_max'] - PC6['N_NW_max'] > PC6['N_WE_min']),
                                       PC6['pop_est'] - PC6['N_NL_max'] - PC6['N_NW_max'])
PC6['N_NW_min'] = PC6['N_NW_min'].mask((PC6['pop_est'] - PC6['N_WE_max'] - PC6['N_NL_max'] > PC6['N_NW_min']),
                                       PC6['pop_est'] - PC6['N_WE_max'] - PC6['N_NL_max'])
PC6['N_NL_max'] = PC6['N_NL_max'].mask((PC6['pop_est'] - PC6['N_WE_min'] - PC6['N_NW_min'] < PC6['N_NL_max']),
                                       PC6['pop_est'] - PC6['N_WE_min'] - PC6['N_NW_min'])
PC6['N_WE_max'] = PC6['N_WE_max'].mask((PC6['pop_est'] - PC6['N_NL_min'] - PC6['N_NW_min'] < PC6['N_WE_max']),
                                       PC6['pop_est'] - PC6['N_NL_min'] - PC6['N_NW_min'])
PC6['N_NW_max'] = PC6['N_NW_max'].mask((PC6['pop_est'] - PC6['N_WE_min'] - PC6['N_NL_min'] < PC6['N_NW_max']),
                                       PC6['pop_est'] - PC6['N_WE_min'] - PC6['N_NL_min'])

In [19]:
PC6['PC5'] = PC6['Postcode'].str.slice(0,5)

In [20]:
PC6['zero_pop'] = 0
PC6['bool_pop'] = 1
PC6['zero_pop'] = PC6['zero_pop'].mask(PC6['pop_est'] == 0,1)

In [21]:
# Computing the statistics for the 5-digits postcodes from the 6-digit postcodes.
PC5_est = PC6.drop(columns = ['P_NL_ACHTG','P_WE_MIG_A', 'P_NW_MIG_A']).groupby('PC5').sum()

PC5_est = PC5_est.reset_index()

PC5_est = PC5_est.rename(columns = {'pop_est':'pop_est_PC6_agg',
                                    'pop':'pop_PC6_agg',
                                    'N_NL_min':'N_NL_min_PC6_agg',
                                    'N_NL_max':'N_NL_max_PC6_agg',
                                    'N_WE_min':'N_WE_min_PC6_agg',
                                    'N_WE_max':'N_WE_max_PC6_agg',
                                    'N_NW_min':'N_NW_min_PC6_agg',
                                    'N_NW_max':'N_NW_max_PC6_agg'})

In [22]:
PC5 = gpd.read_file('../data/raw_data/sociodemographics/PC5/CBS_PC5_2017_v3.shp')

PC5 = PC5.loc[:,['PC5','INWONER','P_NL_ACHTG', 'P_WE_MIG_A','P_NW_MIG_A','MAN', 'VROUW',
                 'INW_014','INW_1524','INW_2544', 'INW_4564','INW_65PL','AANTAL_HH','TOTHH_EENP',
                 'TOTHH_MPZK','HH_EENOUD','HH_TWEEOUD','GEM_HH_GR','geometry']]

PC5 = PC5.rename(columns = {'INWONER':'pop',
                            'MAN':'pop_M',
                            'VROUW':'pop_V',
                            'INW_014':'pop_014',
                            'INW_1524':'pop_1524',
                            'INW_2544':'pop_2544',
                            'INW_4564':'pop_4564',
                            'INW_65PL':'pop_65PL',
                            'AANTAL_HH':'N_HH',
                            'TOTHH_EENP':'HH1',
                            'TOTHH_MPZK':'HH2',
                            'HH_EENOUD':'HH3',
                            'HH_TWEEOUD':'HH4',
                            'GEM_HH_GR':'pop_per_HH'})

PC5['N_HH_ori'] = PC5['N_HH'].copy()
PC5['pop_per_HH_ori'] = PC5['pop_per_HH'].copy()

PC5 = PC5.replace(-99997,0)

PC5.loc[:,'pop_014'] = PC5['pop_014'].mask((PC5['pop_014'] == 0) & (PC5['pop'] != 0), 2)
PC5.loc[:,'pop_1524'] = PC5['pop_1524'].mask((PC5['pop_1524'] == 0) & (PC5['pop'] != 0), 2)
PC5.loc[:,'pop_2544'] = PC5['pop_2544'].mask((PC5['pop_2544'] == 0) & (PC5['pop'] != 0), 2)
PC5.loc[:,'pop_4564'] = PC5['pop_4564'].mask((PC5['pop_4564'] == 0) & (PC5['pop'] != 0), 2)
PC5.loc[:,'pop_65PL'] = PC5['pop_65PL'].mask((PC5['pop_65PL'] == 0) & (PC5['pop'] != 0), 2)
PC5.loc[:,'pop_M'] = PC5['pop_M'].mask((PC5['pop_M'] == 0) & (PC5['pop'] != 0), 2)
PC5.loc[:,'pop_V'] = PC5['pop_V'].mask((PC5['pop_V'] == 0) & (PC5['pop'] != 0), 2)
PC5.loc[:,'HH1'] = PC5['HH1'].mask((PC5['HH1'] == 0) & (PC5['N_HH'] != 0), 2)
PC5.loc[:,'HH2'] = PC5['HH2'].mask((PC5['HH2'] == 0) & (PC5['N_HH'] != 0), 2)
PC5.loc[:,'HH3'] = PC5['HH3'].mask((PC5['HH3'] == 0) & (PC5['N_HH'] != 0), 2)
PC5.loc[:,'HH4'] = PC5['HH4'].mask((PC5['HH4'] == 0) & (PC5['N_HH'] != 0), 2)

In [23]:
PC5 = proba_age_pop(PC5,'pop','pop_014','pop_1524', 'pop_2544','pop_4564','pop_65PL')

PC5 = proba_gender_pop(PC5, 'pop','pop_M','pop_V')

PC5 = proba_hh(PC5,'N_HH','HH1','HH2','HH3','HH4') 

PC5 = proba_hh_pop(PC5, 'pop', 'N_HH','pop_per_HH','Phh_n2','Phh_n1','Phh_0','Phh_p1','Phh_p2')

PC5['Pn2'] = PC5['Pa_n2'] * PC5['Pg_n2'] * PC5['Ph_n2'] 
PC5['Pn1'] = PC5['Pa_n1'] * PC5['Pg_n1'] * PC5['Ph_n1'] 
PC5['P0'] = PC5['Pa_0'] * PC5['Pg_0'] * PC5['Ph_0'] 
PC5['Pp1'] = PC5['Pa_p1'] * PC5['Pg_p1'] * PC5['Ph_p1'] 
PC5['Pp2'] = PC5['Pa_p2'] * PC5['Pg_p2'] * PC5['Ph_p2'] 

PC5['P_tot'] = PC5[['Pn2','Pn1','P0','Pp1','Pp2']].sum(axis = 1)

PC5['Pn2'] = PC5['Pn2'].mask(PC5['P_tot'] == 0, PC5['Pa_n2'] * PC5['Pg_n2'])
PC5['Pn1'] = PC5['Pn1'].mask(PC5['P_tot'] == 0, PC5['Pa_n1'] * PC5['Pg_n1'])
PC5['P0']  = PC5['P0'].mask(PC5['P_tot'] == 0, PC5['Pa_0'] * PC5['Pg_0'])
PC5['Pp1'] = PC5['Pp1'].mask(PC5['P_tot'] == 0, PC5['Pa_p1'] * PC5['Pg_p1'])
PC5['Pp2'] = PC5['Pp2'].mask(PC5['P_tot'] == 0, PC5['Pa_p2'] * PC5['Pg_p2'])

PC5['P_tot'] = PC5['P_tot'].mask(PC5['P_tot'] == 0,  PC5[['Pn2','Pn1','P0','Pp1','Pp2']].sum(axis = 1))

PC5['Pn2'] = PC5['Pn2'] / PC5['P_tot']
PC5['Pn1'] = PC5['Pn1'] / PC5['P_tot']
PC5['P0']  = PC5['P0']  / PC5['P_tot']
PC5['Pp1'] = PC5['Pp1'] / PC5['P_tot']
PC5['Pp2'] = PC5['Pp2'] / PC5['P_tot']

PC5['pop_est'] = (PC5['pop'] - 2) * PC5['Pn2'] + \
                 (PC5['pop'] - 1) * PC5['Pn1'] + \
                 (PC5['pop']    ) * PC5['P0']  + \
                 (PC5['pop'] + 1) * PC5['Pp1'] + \
                 (PC5['pop'] + 2) * PC5['Pp2']
PC5['pop_est'] = PC5['pop_est'].round().astype(int)

In [24]:
PC5 = PC5.loc[:,['PC5','pop_est','P_NL_ACHTG', 'P_WE_MIG_A', 'P_NW_MIG_A','pop']]

In [25]:
PC5['N_NL_min'] = (PC5['pop_est'] * (PC5['P_NL_ACHTG']-5) /100).round()
PC5['N_NL_min'] = PC5['N_NL_min'].mask(PC5['N_NL_min'] < 0,0)
PC5['N_NL_min'] = PC5['N_NL_min'].mask((PC5['N_NL_min'] < 5) & (PC5['P_NL_ACHTG'] > 0),5)
PC5['N_NL_max'] = (PC5['pop_est'] * (PC5['P_NL_ACHTG'] + 5) /100).round()
PC5['N_NL_max'] = PC5['N_NL_max'].mask((PC5['N_NL_max'] < 5) & (PC5['pop_est']>0),4)
PC5['N_NL_max'] = PC5['N_NL_max'].mask(PC5['N_NL_max'] > PC5['pop_est'],PC5['pop_est'])

In [26]:
PC5['N_WE_min'] = (PC5['pop_est'] * (PC5['P_WE_MIG_A'] - 5) /100).round()
PC5['N_WE_min'] = PC5['N_WE_min'].mask(PC5['N_WE_min'] < 0, 0)
PC5['N_WE_min'] = PC5['N_WE_min'].mask((PC5['N_WE_min'] < 5) & (PC5['P_WE_MIG_A'] > 0),5)
PC5['N_WE_max'] = (PC5['pop_est'] * (PC5['P_WE_MIG_A'] + 5) /100).round()
PC5['N_WE_max'] = PC5['N_WE_max'].mask((PC5['N_WE_max'] < 5) & (PC5['pop_est'] > 0),4)
PC5['N_WE_max'] = PC5['N_WE_max'].mask(PC5['N_WE_max'] > PC5['pop_est'],PC5['pop_est'])

In [27]:
PC5['N_NW_min'] = (PC5['pop_est'] * (PC5['P_NW_MIG_A'] - 5) /100).round()
PC5['N_NW_min'] = PC5['N_NW_min'].mask(PC5['N_NW_min'] < 0,0)
PC5['N_NW_min'] = PC5['N_NW_min'].mask((PC5['N_NW_min'] < 5) &  (PC5['P_NW_MIG_A'] > 0),5)
PC5['N_NW_max'] = (PC5['pop_est'] * (PC5['P_NW_MIG_A'] + 5) /100).round()
PC5['N_NW_max'] = PC5['N_NW_max'].mask((PC5['N_NW_max'] < 5) & (PC5['pop_est'] > 0),4)
PC5['N_NW_max'] = PC5['N_NW_max'].mask(PC5['N_NW_max'] > PC5['pop_est'], PC5['pop_est'])

In [28]:
PC5['pop_est']=PC5['pop_est'].mask(PC5['N_NL_min'] + PC5['N_WE_min'] + PC5['N_NW_min'] > PC5['pop_est'],
                                   PC5['N_NL_min'] + PC5['N_WE_min'] + PC5['N_NW_min'])
PC5['pop_est']=PC5['pop_est'].mask((PC5['N_NL_max'] + PC5['N_WE_max'] + PC5['N_NW_max'] < PC5['pop_est']) & 
                                   (PC5['P_NL_ACHTG'] + PC5['P_WE_MIG_A'] + PC5['P_NW_MIG_A'] > 0),
                                   PC5['N_NL_max'] + PC5['N_WE_max'] + PC5['N_NW_max'])
PC5['N_NW_max'] = PC5['N_NW_max'].mask((PC5['N_NL_max'] + PC5['N_WE_max'] + PC5['N_NW_max'] < PC5['pop_est']) & 
                                       (PC5['P_NL_ACHTG'] + PC5['P_WE_MIG_A'] + PC5['P_NW_MIG_A'] == 0),
                                       PC5['pop_est'])
PC5['N_NL_max'] = PC5['N_NL_max'].mask((PC5['N_NL_max'] + PC5['N_WE_max']+ PC5['N_NW_max'] < PC5['pop_est']) & 
                                       (PC5['P_NL_ACHTG'] + PC5['P_WE_MIG_A'] + PC5['P_NW_MIG_A'] == 0),
                                       PC5['pop_est'])
PC5['N_WE_max'] = PC5['N_WE_max'].mask((PC5['N_NL_max'] + PC5['N_WE_max'] + PC5['N_NW_max'] < PC5['pop_est']) & 
                                       (PC5['P_NL_ACHTG'] + PC5['P_WE_MIG_A'] + PC5['P_NW_MIG_A'] == 0),
                                       PC5['pop_est'])

PC5['N_NL_max'] = PC5['N_NL_max'].mask(PC5['N_NL_min'] > PC5['N_NL_max'], PC5['N_NL_min'])
PC5['N_WE_max'] = PC5['N_WE_max'].mask(PC5['N_WE_min'] > PC5['N_WE_max'], PC5['N_WE_min'])
PC5['N_NW_max'] = PC5['N_NW_max'].mask(PC5['N_NW_min'] > PC5['N_NW_max'], PC5['N_NW_min'])

In [29]:
PC5['N_NL_min'] = PC5['N_NL_min'].mask((PC5['pop_est'] - PC5['N_WE_max'] - PC5['N_NW_max'] > PC5['N_NL_min']),
                                       PC5['pop_est'] - PC5['N_WE_max'] - PC5['N_NW_max'])
PC5['N_WE_min'] = PC5['N_WE_min'].mask((PC5['pop_est'] - PC5['N_NL_max'] - PC5['N_NW_max'] > PC5['N_WE_min']),
                                       PC5['pop_est'] - PC5['N_NL_max'] - PC5['N_NW_max'])
PC5['N_NW_min'] = PC5['N_NW_min'].mask((PC5['pop_est'] - PC5['N_WE_max'] - PC5['N_NL_max'] > PC5['N_NW_min']),
                                       PC5['pop_est'] - PC5['N_WE_max'] - PC5['N_NL_max'])
PC5['N_NL_max'] = PC5['N_NL_max'].mask((PC5['pop_est'] - PC5['N_WE_min'] - PC5['N_NW_min'] < PC5['N_NL_max']),
                                       PC5['pop_est'] - PC5['N_WE_min'] - PC5['N_NW_min'])
PC5['N_WE_max'] = PC5['N_WE_max'].mask((PC5['pop_est'] - PC5['N_NL_min'] - PC5['N_NW_min'] < PC5['N_WE_max']),
                                       PC5['pop_est'] - PC5['N_NL_min'] - PC5['N_NW_min'])
PC5['N_NW_max'] = PC5['N_NW_max'].mask((PC5['pop_est'] - PC5['N_WE_min'] - PC5['N_NL_min'] < PC5['N_NW_max']),
                                       PC5['pop_est'] - PC5['N_WE_min'] - PC5['N_NL_min'])

In [30]:
# Comparing the population data for the 5-digits postcodes to the population for the 6-digits postcodes aggregated
# at the 5-digit level.
PC5 = pd.merge(PC5, PC5_est, on = 'PC5')
PC5 = PC5.drop(columns = ['zero_pop', 'pop', 'pop_PC6_agg'])
PC5['diff'] = PC5['pop_est'] - PC5['pop_est_PC6_agg']

In [31]:
# When the population of the PC5 obtained by aggregating the PC6 is smaller than the actual population of the PC5,
# it means that data is missing at the 6-digits postcode level.
# We correct that by distributing the missing population to the different 6-digits postcodes.

# We focus on the postcodes for which the missing population is larger than 2 person per zone.
# if the missing population is lower than that threshold, the difference could be due to a rounding error.
PC5['pop_est_PC5']  = PC5['pop_est_PC6_agg'].mask(PC5['diff']  / PC5['bool_pop'] > 2, PC5['pop_est'])
PC5['N_NL_min_PC5'] = PC5['N_NL_min_PC6_agg'].mask(PC5['diff'] / PC5['bool_pop'] > 2, PC5['N_NL_min'])
PC5['N_NL_max_PC5'] = PC5['N_NL_max_PC6_agg'].mask(PC5['diff'] / PC5['bool_pop'] > 2, PC5['N_NL_max'])
PC5['N_WE_min_PC5'] = PC5['N_WE_min_PC6_agg'].mask(PC5['diff'] / PC5['bool_pop'] > 2, PC5['N_WE_min'])
PC5['N_WE_max_PC5'] = PC5['N_WE_max_PC6_agg'].mask(PC5['diff'] / PC5['bool_pop'] > 2, PC5['N_WE_max'])
PC5['N_NW_min_PC5'] = PC5['N_NW_min_PC6_agg'].mask(PC5['diff'] / PC5['bool_pop'] > 2, PC5['N_NW_min'])
PC5['N_NW_max_PC5'] = PC5['N_NW_max_PC6_agg'].mask(PC5['diff'] / PC5['bool_pop'] > 2, PC5['N_NW_max'])
PC5['diff'] = PC5['diff'].mask(PC5['diff'] / PC5['bool_pop'] > 2, 0)

In [32]:
# Adjusting the minimums and maximums.
PC5['N_NW_min_PC5'] = PC5['N_NW_min_PC5'].mask(PC5['N_NW_min_PC5'] < PC5['N_NW_min_PC6_agg'], 
                                               PC5['N_NW_min_PC6_agg'])
PC5['N_WE_min_PC5'] = PC5['N_WE_min_PC5'].mask(PC5['N_WE_min_PC5'] < PC5['N_WE_min_PC6_agg'], 
                                               PC5['N_WE_min_PC6_agg'])
PC5['N_NL_min_PC5'] = PC5['N_NL_min_PC5'].mask(PC5['N_NL_min_PC5'] < PC5['N_NL_min_PC6_agg'], 
                                               PC5['N_NL_min_PC6_agg'])

In [33]:
PC5['N_NW_max_PC5'] = PC5['N_NW_max_PC5'].mask(PC5['N_NW_max'] < PC5['N_NW_max_PC5'], PC5['N_NW_max'])
PC5['N_NL_max_PC5'] = PC5['N_NL_max_PC5'].mask(PC5['N_NL_max'] < PC5['N_NL_max_PC5'], PC5['N_NL_max'])
PC5['N_WE_max_PC5'] = PC5['N_WE_max_PC5'].mask(PC5['N_WE_max'] < PC5['N_WE_max_PC5'], PC5['N_WE_max'])
PC5['N_NW_min_PC5'] = PC5['N_NW_min_PC5'].mask(PC5['N_NW_min'] > PC5['N_NW_min_PC5'], PC5['N_NW_min'])
PC5['N_NL_min_PC5'] = PC5['N_NL_min_PC5'].mask(PC5['N_NL_min'] > PC5['N_NL_min_PC5'], PC5['N_NL_min'])
PC5['N_WE_min_PC5'] = PC5['N_WE_min_PC5'].mask(PC5['N_WE_min'] > PC5['N_WE_min_PC5'], PC5['N_WE_min'])

Sometimes, the minimum exceeds the maximum. In this case, we set it to the PC5 level.

In [34]:
PC5['N_WE_max_PC5'] = PC5['N_WE_max_PC5'].mask((PC5['N_WE_max_PC5'] < PC5['N_WE_min_PC5']) &
                                               (PC5['N_WE_min'] >= PC5['N_WE_min_PC5']),
                                               PC5['N_WE_min'])
PC5['N_NW_max_PC5'] = PC5['N_NW_max_PC5'].mask((PC5['N_NW_max_PC5'] < PC5['N_NW_min_PC5']) & 
                                               (PC5['N_NW_min'] >= PC5['N_NW_min_PC5']),
                                               PC5['N_NW_min'])
PC5['N_NL_max_PC5'] = PC5['N_NL_max_PC5'].mask((PC5['N_NL_max_PC5'] < PC5['N_NL_min_PC5']) & 
                                               (PC5['N_NL_min'] >= PC5['N_NL_min_PC5']),
                                               PC5['N_NL_min'])

In [35]:
PC5['N_WE_min_PC6_agg'] = PC5['N_WE_min_PC6_agg'].mask((PC5['N_WE_max_PC5'] < PC5['N_WE_min_PC5']) & 
                                                       (PC5['N_WE_max'] >= PC5['N_WE_max_PC5']),
                                                       PC5['N_WE_max'])
PC5['N_NW_min_PC6_agg'] = PC5['N_NW_min_PC6_agg'].mask((PC5['N_NW_max_PC5'] < PC5['N_NW_min_PC5']) &
                                                       (PC5['N_NW_max'] >= PC5['N_NW_max_PC5']),
                                                       PC5['N_NW_max'])
PC5['N_NL_min_PC6_agg'] = PC5['N_NL_min_PC6_agg'].mask((PC5['N_NL_max_PC5'] < PC5['N_NL_min_PC5']) &
                                                       (PC5['N_NL_max'] >= PC5['N_NL_max_PC5']),
                                                       PC5['N_NL_max'])

PC5['N_WE_min_PC5'] = PC5['N_WE_min_PC5'].mask((PC5['N_WE_max_PC5'] < PC5['N_WE_min_PC5']) & 
                                               (PC5['N_WE_max'] >= PC5['N_WE_max_PC5']),
                                               PC5['N_WE_max'])
PC5['N_NW_min_PC5'] = PC5['N_NW_min_PC5'].mask((PC5['N_NW_max_PC5'] < PC5['N_NW_min_PC5']) &
                                               (PC5['N_NW_max'] >= PC5['N_NW_max_PC5']),
                                               PC5['N_NW_max'])
PC5['N_NL_min_PC5'] = PC5['N_NL_min_PC5'].mask((PC5['N_NL_max_PC5'] < PC5['N_NL_min_PC5']) &
                                               (PC5['N_NL_max'] >= PC5['N_NL_max_PC5']),
                                               PC5['N_NL_max'])

In [36]:
PC5 = PC5.drop(columns = ['N_NL_min','N_NL_max','N_WE_min','N_WE_max',
                          'N_NW_min','N_NW_max','bool_pop','diff','pop_est'])

In [37]:
PC5 = PC5.rename(columns = {'P_NL_ACHTG':'P_NL_PC5',
                            'P_WE_MIG_A':'P_WE_PC5',
                            'P_NW_MIG_A':'P_NW_PC5'})

In [38]:
# PC6_won contains the number of housing per zone.
# The number of housing is used to allocate the missing population among zones that have no population indicated.
PC6_won['PC5'] = PC6_won['Postcode'].str.slice(0,5)
PC6_won = PC6_won.replace(-99997,0)

PC6_edited = pd.merge(PC6, PC5, on = 'PC5')
PC6_edited = pd.merge(PC6_edited, PC6_won.drop(columns = 'PC5'), on = 'Postcode')
PC6_edited['diff'] = PC6_edited['pop_est_PC5'] - PC6_edited['pop_est_PC6_agg']

# We consider only the housing in which population is missing, 
# and in which the population in the PC6 is given as 0, while there is housing in them.
ratio_woning = PC6_edited.loc[(PC6_edited['pop_est_PC5'] > PC6_edited['pop_est_PC6_agg']) & 
                              (PC6_edited['WONING'] > 0) & 
                              (PC6_edited['pop_est'] == 0)][['Postcode','PC5','WONING']].copy()

# Computing the total number of housing in the PC5.
tot_woning = ratio_woning[['PC5','WONING']].groupby('PC5').sum().reset_index()

tot_woning = tot_woning.rename(columns = {'WONING':'tot_woning'})

ratio_woning = pd.merge(ratio_woning,
                        tot_woning,
                        on = 'PC5')

# Computing the proportion of housing of the PC5 located in each PC6 belonging to the PC5.
ratio_woning['ratio_woning'] = ratio_woning['WONING'] / ratio_woning['tot_woning']

PC6_edited = pd.merge(PC6_edited, ratio_woning[['Postcode','ratio_woning']],
                      on = 'Postcode', how = 'left')

PC6_edited['ratio_woning'] = PC6_edited['ratio_woning'].mask(PC6_edited['ratio_woning'].isna(),0)

PC6_edited['pop_est'] = PC6_edited['pop_est'].mask((PC6_edited['pop_est_PC5'] > PC6_edited['pop_est_PC6_agg']) & 
                                                   (PC6_edited['WONING'] > 0) & (PC6_edited['pop_est'] == 0), 
                                                   (PC6_edited['pop_est_PC5'] - PC6_edited['pop_est_PC6_agg']) * PC6_edited['ratio_woning'])

PC6_edited['diff'] = PC6_edited['diff'].mask(PC6_edited['PC5'].isin(ratio_woning['PC5']), 0)

In [39]:
# Flag indicating when the population has been changed.
PC6_edited['modif'] = 0
PC6_edited['modif'] = PC6_edited['modif'].mask(PC6_edited['ratio_woning']>0,1)
PC6_edited['pop_est'] = PC6_edited['pop_est'].round()

In [40]:
PC6_edited = PC6_edited.drop(columns = ['pop','WONING','ratio_woning'])

While the code above allows to distribute the population when the housing variable is defined, in some other zones this variable is not defined. In this situation, we allocate by hand the population. We only consider the cases where the zones are located in densely populated areas. The list below shows the zones that have been investigated.

- 1102S: all in 1102SL
- 1213N: dismiss
- 1625M: all in 1625MD
- 1741L: all in 1741LW
- 2291P: all in 2291PW
- 2333Y: evenly distributed
- 2402N: all in 2402NM
- 2402W: 2402WC and 2402WB
- 2404C: dismiss (probably prison in 2404CA)
- 2408Z: split between 2408ZH and 2408ZB 
- 2624G: all in 2624GA
- 2967L: split between LA and LB
- 3192W: everything in 3192WE
- 3584Z: dismiss
- 3734A: evenly distributed among empty zones
- 6431A: split entre 6431AE et 6431AZ.
- 8103J: split between 8103JA and JB
- 8225C: all in 8225CB
- 8561D: all in 8561DD
- 9206A: dismiss
- 9617A: dismiss.
- 9801W: split between 9801WK and WP
- 1681P: distribute evenly accross all zones
- 5971E: all in 5971ED 
- 3087C: all in 3087CJ
- 1336J: all in 1336JC


In [41]:
PC6_edited = PC6_edited.set_index('Postcode')

In [42]:
PC6_edited.loc['6431AE','pop_est'] = PC6_edited.loc['6431AE','pop_est'] + (PC6_edited.loc['6431AE','diff']/2).round()
PC6_edited.loc['6431AZ','pop_est'] = PC6_edited.loc['6431AZ','pop_est'] + (PC6_edited.loc['6431AZ','diff']/2).round()
PC6_edited.loc[PC6_edited['PC5']=='6431A','diff'] = 0
PC6_edited.loc[['6431AE','6431AZ'],'modif'] = 1

PC6_edited.loc['1625MD','pop_est'] = PC6_edited.loc['1625MD','pop_est'] + PC6_edited.loc['1625MD','diff']
PC6_edited.loc[PC6_edited['PC5']=='1625M','diff'] = 0
PC6_edited.loc['1625MD','modif'] = 1

PC6_edited.loc['2402NM','pop_est'] = PC6_edited.loc['2402NM','pop_est'] + PC6_edited.loc['2402NM','diff']
PC6_edited.loc[PC6_edited['PC5']=='2402N','diff'] = 0
PC6_edited.loc['2402NM','modif'] = 1

PC6_edited.loc['8103JA','pop_est'] = PC6_edited.loc['8103JA','pop_est'] + (PC6_edited.loc['8103JA','diff']/2).round()
PC6_edited.loc['8103JB','pop_est'] = PC6_edited.loc['8103JB','pop_est'] + (PC6_edited.loc['8103JB','diff']/2).round()
PC6_edited.loc[PC6_edited['PC5']=='8103J','diff'] = 0
PC6_edited.loc[['6431AE','6431AZ'],'modif'] = 1

PC6_edited.loc['3192WE','pop_est'] = PC6_edited.loc['3192WE','pop_est'] + PC6_edited.loc['3192WE','diff']
PC6_edited.loc[PC6_edited['PC5']=='3192W','diff'] = 0
PC6_edited.loc['3192WE','modif'] = 1


PC6_edited.loc['8103JA','pop_est'] = PC6_edited.loc['8103JA','pop_est'] + (PC6_edited.loc['8103JA','diff']/2).round()
PC6_edited.loc['8103JB','pop_est'] = PC6_edited.loc['8103JB','pop_est'] + (PC6_edited.loc['8103JB','diff']/2).round()
PC6_edited.loc[PC6_edited['PC5']=='2402N','diff'] = 0
PC6_edited.loc[['8103JA','8103JB'],'modif'] = 1


PC6_edited.loc[(PC6_edited['PC5'] == '3734A') & 
         (PC6_edited['pop_est'] == 0),['pop_est']] = (PC6_edited.loc['3734AB','diff']/len(PC6_edited.loc[(PC6_edited['PC5'] == '3734A') & 
                                                                                      (PC6_edited['pop_est'] == 0)])).round()
PC6_edited.loc[PC6_edited['PC5']=='3734A','diff'] = 0
PC6_edited.loc[(PC6_edited['PC5']=='3734A') & (PC6_edited['pop_est'] == 0),'modif'] = 1

PC6_edited.loc['8561DD','pop_est'] = PC6_edited.loc['8561DD','pop_est'] + PC6_edited.loc['8561DD','diff']
PC6_edited.loc[PC6_edited['PC5']=='8561D','diff'] = 0
PC6_edited.loc['8561DD','modif'] = 1

PC6_edited.loc['2624GA','pop_est'] = PC6_edited.loc['2624GA','pop_est'] + PC6_edited.loc['2624GA','diff']
PC6_edited.loc[PC6_edited['PC5']=='2624G','diff'] = 0
PC6_edited.loc['2624GA','modif'] = 1

PC6_edited.loc['2291PW','pop_est'] = PC6_edited.loc['2291PW','pop_est'] + PC6_edited.loc['2291PW','diff']
PC6_edited.loc[PC6_edited['PC5']=='2291P','diff'] = 0
PC6_edited.loc['2291PW','modif'] = 1

PC6_edited.loc[(PC6_edited['PC5'] == '2333Z'),['pop_est']] = (PC6_edited.loc['2333ZA','diff']/len(PC6_edited.loc[PC6_edited['PC5'] == '2333Z'])).round()
PC6_edited.loc[PC6_edited['PC5']=='2333Z','diff'] = 0
PC6_edited.loc[PC6_edited['PC5']=='2333Z','modif'] = 1

PC6_edited.loc['2408ZH','pop_est'] = PC6_edited.loc['2408ZH','pop_est'] + (PC6_edited.loc['2408ZH','diff']/2).round()
PC6_edited.loc['2408ZB','pop_est'] = PC6_edited.loc['2408ZB','pop_est'] + (PC6_edited.loc['2408ZB','diff']/2).round()
PC6_edited.loc[PC6_edited['PC5']=='2408Z','diff'] = 0
PC6_edited.loc[['2408ZH','2408ZB'],'modif'] = 1

PC6_edited.loc['2402WC','pop_est'] = PC6_edited.loc['2402WC','pop_est'] + (PC6_edited.loc['2402WC','diff']/2).round()
PC6_edited.loc['2402WB','pop_est'] = PC6_edited.loc['2402WB','pop_est'] + (PC6_edited.loc['2402WB','diff']/2).round()
PC6_edited.loc[PC6_edited['PC5']=='2402W','diff'] = 0
PC6_edited.loc[['2402WC','2402WB'],'modif'] = 1

PC6_edited.loc['2967LA','pop_est'] = PC6_edited.loc['2967LA','pop_est'] + (PC6_edited.loc['2967LA','diff']/2).round()
PC6_edited.loc['2967LB','pop_est'] = PC6_edited.loc['2967LB','pop_est'] + (PC6_edited.loc['2967LB','diff']/2).round()
PC6_edited.loc[PC6_edited['PC5']=='2967L','diff'] = 0
PC6_edited.loc[['2967LA','2967LB'],'modif'] = 1

PC6_edited.loc['1102SL','pop_est'] = PC6_edited.loc['1102SL','pop_est'] + PC6_edited.loc['1102SL','diff']
PC6_edited.loc[PC6_edited['PC5']=='1102S','diff'] = 0
PC6_edited.loc['1102SL','modif'] = 1

PC6_edited.loc['1741LW','pop_est'] = PC6_edited.loc['1741LW','pop_est'] + PC6_edited.loc['1741LW','diff']
PC6_edited.loc[PC6_edited['PC5']=='1741L','diff'] = 0
PC6_edited.loc['1741LW','modif'] = 1

PC6_edited.loc['8225CB','pop_est'] = PC6_edited.loc['8225CB','pop_est'] + PC6_edited.loc['8225CB','diff']
PC6_edited.loc[PC6_edited['PC5']=='8225C','diff'] = 0
PC6_edited.loc['8225CB','modif'] = 1

PC6_edited.loc['9801WK','pop_est'] = PC6_edited.loc['9801WK','pop_est'] + (PC6_edited.loc['9801WK','diff']/2).round()
PC6_edited.loc['9801WP','pop_est'] = PC6_edited.loc['9801WP','pop_est'] + (PC6_edited.loc['9801WP','diff']/2).round()
PC6_edited.loc[PC6_edited['PC5']=='9801W','diff'] = 0
PC6_edited.loc[['9801WK','9801WP'],'modif'] = 1

PC6_edited.loc[(PC6_edited['PC5'] == '1681P'),['pop_est']] = (PC6_edited.loc['1681PA','diff']/len(PC6_edited.loc[(PC6_edited['PC5'] == '1681P')])).round()
PC6_edited.loc[PC6_edited['PC5']=='1681P','diff'] = 0
PC6_edited.loc[PC6_edited['PC5']=='1681P','modif'] = 1

PC6_edited.loc['5971ED','pop_est'] = PC6_edited.loc['5971ED','pop_est'] + PC6_edited.loc['5971ED','diff']
PC6_edited.loc[PC6_edited['PC5']=='5971E','diff'] = 0
PC6_edited.loc['1625MD','modif'] = 1

PC6_edited.loc['3087CJ','pop_est'] = PC6_edited.loc['3087CJ','pop_est'] + PC6_edited.loc['3087CJ','diff']
PC6_edited.loc[PC6_edited['PC5']=='3087C','diff'] = 0
PC6_edited.loc['3087CJ','modif'] = 1

PC6_edited.loc['1336JC','pop_est'] = PC6_edited.loc['1336JC','pop_est'] + PC6_edited.loc['1336JC','diff']
PC6_edited.loc[PC6_edited['PC5']=='1336J','diff'] = 0
PC6_edited.loc['1336JC','modif'] = 1

In [43]:
# Population for which the social group is unknown.
PC6_edited['undefined_PC5'] = PC6_edited['pop_est_PC5'] - PC6_edited['N_NL_min_PC5'] - PC6_edited['N_WE_min_PC5'] - PC6_edited['N_NW_min_PC5']

In [44]:
PC6_edited['pop_est_PC5']  = PC6_edited['pop_est_PC5'].mask(PC6_edited['undefined_PC5'] < 0, PC6_edited['pop_est_PC6_agg'])
PC6_edited['N_NL_min_PC5'] = PC6_edited['N_NL_min_PC5'].mask(PC6_edited['undefined_PC5'] < 0, PC6_edited['N_NL_min_PC6_agg'])
PC6_edited['N_NL_max_PC5'] = PC6_edited['N_NL_max_PC5'].mask(PC6_edited['undefined_PC5'] < 0, PC6_edited['N_NL_max_PC6_agg'])
PC6_edited['N_WE_min_PC5'] = PC6_edited['N_WE_min_PC5'].mask(PC6_edited['undefined_PC5'] < 0, PC6_edited['N_WE_min_PC6_agg'])
PC6_edited['N_WE_max_PC5'] = PC6_edited['N_WE_max_PC5'].mask(PC6_edited['undefined_PC5'] < 0, PC6_edited['N_WE_max_PC6_agg'])
PC6_edited['N_NW_min_PC5'] = PC6_edited['N_NW_min_PC5'].mask(PC6_edited['undefined_PC5'] < 0, PC6_edited['N_NW_min_PC6_agg'])
PC6_edited['N_NW_max_PC5'] = PC6_edited['N_NW_max_PC5'].mask(PC6_edited['undefined_PC5'] < 0, PC6_edited['N_NW_max_PC6_agg'])

# Population for which the social group is unknown.
PC6_edited['undefined_PC5'] = PC6_edited['pop_est_PC5'] - PC6_edited['N_NL_min_PC5'] - PC6_edited['N_WE_min_PC5'] - PC6_edited['N_NW_min_PC5']

In [45]:
PC6_edited['N_NL'] = PC6_edited['N_NL_min'].copy()
PC6_edited['N_WE'] = PC6_edited['N_WE_min'].copy()
PC6_edited['N_NW'] = PC6_edited['N_NW_min'].copy()

In [46]:
PC6_edited = PC6_edited.reset_index()

In [47]:
pop_modif = PC6_edited.loc[PC6_edited['modif'] == 1, ['pop_est','PC5']].groupby(by = 'PC5').sum().reset_index()
pop_modif = pop_modif.rename(columns = {'pop_est':'pop_modif'})

In [48]:
PC6_edited = pd.merge(PC6_edited, pop_modif,  on = 'PC5', how = 'left')

PC6_edited['pop_modif'] = PC6_edited['pop_modif'].mask(PC6_edited['modif'] == 0, 0)
# pop_modif is the total population in the PC5 where the PC6 were modified.

In [49]:
PC6_edited['pop_modif_ratio'] = PC6_edited['pop_est'] / PC6_edited['pop_modif']
PC6_edited['pop_modif_ratio'] = PC6_edited['pop_modif_ratio'].mask(PC6_edited['modif'] == 0, 0)

In [50]:
PC6_edited['alloc_NL_PC5'] = PC6_edited['N_NL_min_PC5'] - PC6_edited['N_NL_min_PC6_agg']
PC6_edited['alloc_NW_PC5'] = PC6_edited['N_NW_min_PC5'] - PC6_edited['N_NW_min_PC6_agg']
PC6_edited['alloc_WE_PC5'] = PC6_edited['N_WE_min_PC5'] - PC6_edited['N_WE_min_PC6_agg']

In [51]:
PC6_edited['alloc_tot_PC5'] = PC6_edited['alloc_NL_PC5'] + PC6_edited['alloc_WE_PC5'] + PC6_edited['alloc_NW_PC5']
PC6_edited['alloc_NL_ratio_PC5'] = PC6_edited['alloc_NL_PC5']/PC6_edited['alloc_tot_PC5']
PC6_edited['alloc_WE_ratio_PC5'] = PC6_edited['alloc_WE_PC5']/PC6_edited['alloc_tot_PC5']
PC6_edited['alloc_NW_ratio_PC5'] = PC6_edited['alloc_NW_PC5']/PC6_edited['alloc_tot_PC5']

In [52]:
PC6_edited['alloc_NL_ratio_PC5'] = PC6_edited['alloc_NL_ratio_PC5'].mask(PC6_edited['alloc_tot_PC5'] == 0, 0)
PC6_edited['alloc_WE_ratio_PC5'] = PC6_edited['alloc_WE_ratio_PC5'].mask(PC6_edited['alloc_tot_PC5'] == 0, 0)
PC6_edited['alloc_NW_ratio_PC5'] = PC6_edited['alloc_NW_ratio_PC5'].mask(PC6_edited['alloc_tot_PC5'] == 0, 0)

In [53]:
PC6_edited['ratio_determined'] = PC6_edited['alloc_tot_PC5'] / PC6_edited['pop_modif']
PC6_edited['ratio_determined'] = PC6_edited['ratio_determined'].mask(PC6_edited['pop_modif'] == 0,0)
PC6_edited['alloc_tot_PC5'] = PC6_edited['alloc_tot_PC5'].mask((PC6_edited['ratio_determined']<0.7) & (PC6_edited['modif'] == 1),0)
PC6_edited['N_NL_min'] = PC6_edited['N_NL_min'].mask((PC6_edited['ratio_determined']<0.7) & (PC6_edited['modif'] == 1),0)
PC6_edited['N_WE_min'] = PC6_edited['N_WE_min'].mask((PC6_edited['ratio_determined']<0.7) & (PC6_edited['modif'] == 1),0)
PC6_edited['N_NW_min'] = PC6_edited['N_NW_min'].mask((PC6_edited['ratio_determined']<0.7) & (PC6_edited['modif'] == 1),0)
PC6_edited['N_NL_max'] = PC6_edited['N_NL_max'].mask((PC6_edited['ratio_determined']<0.7) & (PC6_edited['modif'] == 1),0)
PC6_edited['N_WE_max'] = PC6_edited['N_WE_max'].mask((PC6_edited['ratio_determined']<0.7) & (PC6_edited['modif'] == 1),0)
PC6_edited['N_NW_max'] = PC6_edited['N_NW_max'].mask((PC6_edited['ratio_determined']<0.7) & (PC6_edited['modif'] == 1),0)

PC6_edited['pop_est'] = PC6_edited['pop_est'].mask((PC6_edited['ratio_determined']<0.7) & (PC6_edited['modif'] == 1),0)

In [54]:
PC6_edited['alloc_tot_PC6'] = PC6_edited[['alloc_tot_PC5','pop_modif']].min(axis = 1)
PC6_edited['alloc_tot_PC6'] = PC6_edited['alloc_tot_PC6'] * PC6_edited['pop_modif_ratio']

PC6_edited['alloc_NL_PC6'] = (PC6_edited['alloc_tot_PC6'] * PC6_edited['alloc_NL_ratio_PC5']).round()
PC6_edited['alloc_NW_PC6'] = (PC6_edited['alloc_tot_PC6'] * PC6_edited['alloc_NW_ratio_PC5']).round()
PC6_edited['alloc_WE_PC6'] = (PC6_edited['alloc_tot_PC6'] * PC6_edited['alloc_WE_ratio_PC5']).round()

PC6_edited['N_NL'] = PC6_edited['N_NL'] + PC6_edited['alloc_NL_PC6']
PC6_edited['N_WE'] = PC6_edited['N_WE'] + PC6_edited['alloc_WE_PC6']
PC6_edited['N_NW'] = PC6_edited['N_NW'] + PC6_edited['alloc_NW_PC6']

In [55]:
PC6_edited['N_NL_min'] = PC6_edited['N_NL_min'].mask(PC6_edited['N_NL_min'] < PC6_edited['N_NL'],PC6_edited['N_NL'])
PC6_edited['N_WE_min'] = PC6_edited['N_WE_min'].mask(PC6_edited['N_WE_min'] < PC6_edited['N_WE'],PC6_edited['N_WE'])
PC6_edited['N_NW_min'] = PC6_edited['N_NW_min'].mask(PC6_edited['N_NW_min'] < PC6_edited['N_NW'],PC6_edited['N_NW'])

PC6_edited['N_NL_max'] = PC6_edited['N_NL_max'].mask(PC6_edited['N_NL_max'] < PC6_edited['N_NL'],PC6_edited['N_NL'])
PC6_edited['N_WE_max'] = PC6_edited['N_WE_max'].mask(PC6_edited['N_WE_max'] < PC6_edited['N_WE'],PC6_edited['N_WE'])
PC6_edited['N_NW_max'] = PC6_edited['N_NW_max'].mask(PC6_edited['N_NW_max'] < PC6_edited['N_NW'],PC6_edited['N_NW'])

In [56]:
PC6_edited['pop_est'] = PC6_edited['pop_est'].mask(PC6_edited['N_NL_min'] + PC6_edited['N_WE_min'] + PC6_edited['N_NW_min'] > PC6_edited['pop_est'],
                                                   PC6_edited['N_NL_min'] + PC6_edited['N_WE_min'] + PC6_edited['N_NW_min'])

In [57]:
PC6_edited['N_NL_max_temp'] = PC6_edited['N_NL_max'].mask(PC6_edited['N_NL_max'] + PC6_edited['N_WE_max'] + PC6_edited['N_NW_max'] - PC6_edited['pop_est'] < 0,
                                                          PC6_edited['pop_est'] - PC6_edited['N_WE_min'] + PC6_edited['N_NW_min'])
PC6_edited['N_WE_max_temp'] = PC6_edited['N_WE_max'].mask(PC6_edited['N_NL_max'] + PC6_edited['N_WE_max'] + PC6_edited['N_NW_max'] - PC6_edited['pop_est'] < 0,
                                                          PC6_edited['pop_est'] - PC6_edited['N_NL_min'] + PC6_edited['N_NW_min'])
PC6_edited['N_NW_max_temp'] = PC6_edited['N_NW_max'].mask(PC6_edited['N_NL_max'] + PC6_edited['N_WE_max'] + PC6_edited['N_NW_max'] - PC6_edited['pop_est'] < 0,
                                                          PC6_edited['pop_est'] - PC6_edited['N_WE_min'] + PC6_edited['N_NL_min'])

PC6_edited['N_NL_max'] = PC6_edited['N_NL_max_temp'].mask(PC6_edited['N_NL_max'] != PC6_edited['N_NL_max_temp'],PC6_edited['N_NL_max_temp'])
PC6_edited['N_WE_max'] = PC6_edited['N_WE_max_temp'].mask(PC6_edited['N_WE_max'] != PC6_edited['N_WE_max_temp'],PC6_edited['N_WE_max_temp'])
PC6_edited['N_NW_max'] = PC6_edited['N_NW_max_temp'].mask(PC6_edited['N_NW_max'] != PC6_edited['N_NW_max_temp'],PC6_edited['N_NW_max_temp'])

In [58]:
PC6_edited['N_NL_min'] = PC6_edited['N_NL_min'].mask((PC6_edited['pop_est'] - PC6_edited['N_WE_max'] - PC6_edited['N_NW_max'] > PC6_edited['N_NL_min']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_WE_max'] - PC6_edited['N_NW_max'])
PC6_edited['N_WE_min'] = PC6_edited['N_WE_min'].mask((PC6_edited['pop_est'] - PC6_edited['N_NL_max'] - PC6_edited['N_NW_max'] > PC6_edited['N_WE_min']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_NL_max'] - PC6_edited['N_NW_max'])
PC6_edited['N_NW_min'] = PC6_edited['N_NW_min'].mask((PC6_edited['pop_est'] - PC6_edited['N_WE_max'] - PC6_edited['N_NL_max'] > PC6_edited['N_NW_min']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_WE_max'] - PC6_edited['N_NL_max'])
PC6_edited['N_NL_max'] = PC6_edited['N_NL_max'].mask((PC6_edited['pop_est'] - PC6_edited['N_WE_min'] - PC6_edited['N_NW_min'] < PC6_edited['N_NL_max']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_WE_min'] - PC6_edited['N_NW_min'])
PC6_edited['N_WE_max'] = PC6_edited['N_WE_max'].mask((PC6_edited['pop_est'] - PC6_edited['N_NL_min'] - PC6_edited['N_NW_min'] < PC6_edited['N_WE_max']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_NL_min'] - PC6_edited['N_NW_min'])
PC6_edited['N_NW_max'] = PC6_edited['N_NW_max'].mask((PC6_edited['pop_est'] - PC6_edited['N_WE_min'] - PC6_edited['N_NL_min'] < PC6_edited['N_NW_max']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_WE_min'] - PC6_edited['N_NL_min'])

In [59]:
PC6_edited = PC6_edited.drop(columns = ['zero_pop', 'bool_pop','pop_est_PC6_agg', 'N_NL_min_PC6_agg', 'N_NL_max_PC6_agg',
                                        'N_WE_min_PC6_agg', 'N_WE_max_PC6_agg', 'N_NW_min_PC6_agg','N_NW_max_PC6_agg',
                                        'pop_est_PC5', 'N_NL_min_PC5', 'N_NL_max_PC5', 'N_WE_min_PC5', 'N_WE_max_PC5',
                                        'N_NW_min_PC5', 'N_NW_max_PC5', 'diff', 'modif', 'undefined_PC5','pop_modif',
                                        'pop_modif_ratio', 'alloc_NL_PC5', 'alloc_NW_PC5', 'alloc_WE_PC5','alloc_tot_PC5', 
                                        'alloc_NL_ratio_PC5', 'alloc_WE_ratio_PC5', 'alloc_NW_ratio_PC5', 'ratio_determined',
                                        'alloc_tot_PC6','alloc_NL_PC6', 'alloc_NW_PC6', 'alloc_WE_PC6', 'N_NL_max_temp',
                                        'N_WE_max_temp', 'N_NW_max_temp' ])

In [60]:
PC6_edited['N_NW_max_temp'] = PC6_edited['N_NW_max'].mask((PC6_edited['N_NW_min'] == 0) & 
                                                          (PC6_edited['N_NW_max'] <= 4) & 
                                                          (PC6_edited['N_NL_max'] + PC6_edited['N_WE_max'] >= 
                                                           PC6_edited['pop_est']),
                                                          0)
PC6_edited['N_WE_max_temp'] = PC6_edited['N_WE_max'].mask((PC6_edited['N_WE_min'] == 0) & 
                                                          (PC6_edited['N_WE_max'] <= 4) & 
                                                          (PC6_edited['N_NL_max'] + PC6_edited['N_NW_max'] >=
                                                           PC6_edited['pop_est']), 
                                                          0)
PC6_edited['N_NL_max_temp'] = PC6_edited['N_NL_max'].mask((PC6_edited['N_NL_min'] == 0) & 
                                                          (PC6_edited['N_NL_max'] <= 4) & 
                                                          (PC6_edited['N_NW_max'] + PC6_edited['N_WE_max'] >= 
                                                           PC6_edited['pop_est']),
                                                          0)

In [61]:
PC6_edited['N_NW_max'] = PC6_edited['N_NW_max'].mask(PC6_edited['N_NL_max_temp'] + PC6_edited['N_WE_max_temp'] >= 
                                                     PC6_edited['pop_est'],
                                                     PC6_edited['N_NW_max_temp'])
PC6_edited['N_WE_max'] = PC6_edited['N_WE_max'].mask(PC6_edited['N_NL_max_temp'] + PC6_edited['N_NW_max_temp'] >=
                                                     PC6_edited['pop_est'],
                                                     PC6_edited['N_WE_max_temp'])
PC6_edited['N_NL_max'] = PC6_edited['N_NL_max'].mask(PC6_edited['N_NW_max_temp'] + PC6_edited['N_WE_max_temp'] >=
                                                     PC6_edited['pop_est'],
                                                     PC6_edited['N_NL_max_temp'])

PC6_edited = PC6_edited.drop(columns = ['N_NL_max_temp','N_WE_max_temp','N_NW_max_temp'])

In [62]:
PC6_edited['N_NL_min_temp'] = PC6_edited['N_NL_min'].mask((PC6_edited['N_NW_min']==0) & (PC6_edited['N_WE_min']==0) &
                                                          (PC6_edited['N_NL_min']==0) & (PC6_edited['pop_est']>0), 
                                                          np.floor(PC6_edited['pop_est'] * PC6_edited['P_NL_ACHTG']/100))
PC6_edited['N_WE_min_temp'] = PC6_edited['N_WE_min'].mask((PC6_edited['N_NW_min']==0) & (PC6_edited['N_WE_min']==0) & 
                                                          (PC6_edited['N_NL_min']==0) & (PC6_edited['pop_est']>0), 
                                                          np.floor(PC6_edited['pop_est'] * PC6_edited['P_WE_MIG_A']/100))
PC6_edited['N_NW_min_temp'] = PC6_edited['N_NW_min'].mask((PC6_edited['N_NW_min']==0) & (PC6_edited['N_WE_min']==0) & 
                                                          (PC6_edited['N_NL_min']==0) & (PC6_edited['pop_est']>0), 
                                                          np.floor(PC6_edited['pop_est'] * PC6_edited['P_NW_MIG_A']/100))

PC6_edited['N_NL_min_temp'] = PC6_edited['N_NL_min_temp'].mask(PC6_edited['N_NL_min_temp']>PC6_edited['N_NL_max'],PC6_edited['N_NL_max'])
PC6_edited['N_WE_min_temp'] = PC6_edited['N_WE_min_temp'].mask(PC6_edited['N_WE_min_temp']>PC6_edited['N_WE_max'],PC6_edited['N_WE_max'])
PC6_edited['N_NW_min_temp'] = PC6_edited['N_NW_min_temp'].mask(PC6_edited['N_NW_min_temp']>PC6_edited['N_NW_max'],PC6_edited['N_NW_max'])

PC6_edited['N_NW_min'] = PC6_edited['N_NW_min'].mask(PC6_edited['N_NW_min_temp'] + 
                                                     PC6_edited['N_WE_min_temp'] + 
                                                     PC6_edited['N_NL_min_temp'] < PC6_edited['pop_est'],
                                                     PC6_edited['N_NW_min_temp'])

PC6_edited['N_WE_min'] = PC6_edited['N_WE_min'].mask(PC6_edited['N_NW_min_temp'] + 
                                                     PC6_edited['N_WE_min_temp'] + 
                                                     PC6_edited['N_NL_min_temp'] < PC6_edited['pop_est'],
                                                     PC6_edited['N_WE_min_temp'])

PC6_edited['N_NL_min'] = PC6_edited['N_NL_min'].mask(PC6_edited['N_NW_min_temp'] + 
                                                     PC6_edited['N_WE_min_temp'] + 
                                                     PC6_edited['N_NL_min_temp'] < PC6_edited['pop_est'],
                                                     PC6_edited['N_NL_min_temp'])

PC6_edited = PC6_edited.drop(columns = ['N_NL_min_temp','N_WE_min_temp','N_NW_min_temp',
                                        'P_NL_ACHTG','P_WE_MIG_A','P_NW_MIG_A'])

In [63]:
PC6_edited['N_NL_min'] = PC6_edited['N_NL_min'].mask((PC6_edited['pop_est'] - PC6_edited['N_WE_max'] - PC6_edited['N_NW_max'] > PC6_edited['N_NL_min']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_WE_max'] - PC6_edited['N_NW_max'])
PC6_edited['N_WE_min'] = PC6_edited['N_WE_min'].mask((PC6_edited['pop_est'] - PC6_edited['N_NL_max'] - PC6_edited['N_NW_max'] > PC6_edited['N_WE_min']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_NL_max'] - PC6_edited['N_NW_max'])
PC6_edited['N_NW_min'] = PC6_edited['N_NW_min'].mask((PC6_edited['pop_est'] - PC6_edited['N_WE_max'] - PC6_edited['N_NL_max'] > PC6_edited['N_NW_min']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_WE_max'] - PC6_edited['N_NL_max'])

PC6_edited['N_NL_max'] = PC6_edited['N_NL_max'].mask((PC6_edited['pop_est'] - PC6_edited['N_WE_min'] - PC6_edited['N_NW_min'] < PC6_edited['N_NL_max']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_WE_min'] - PC6_edited['N_NW_min'])
PC6_edited['N_WE_max'] = PC6_edited['N_WE_max'].mask((PC6_edited['pop_est'] - PC6_edited['N_NL_min'] - PC6_edited['N_NW_min'] < PC6_edited['N_WE_max']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_NL_min'] - PC6_edited['N_NW_min'])
PC6_edited['N_NW_max'] = PC6_edited['N_NW_max'].mask((PC6_edited['pop_est'] - PC6_edited['N_WE_min'] - PC6_edited['N_NL_min'] < PC6_edited['N_NW_max']),
                                                     PC6_edited['pop_est'] - PC6_edited['N_WE_min'] - PC6_edited['N_NL_min'])

In [64]:
PC6_edited['undefined_PC6'] = PC6_edited['pop_est'] - PC6_edited['N_NL_min'] - PC6_edited['N_WE_min'] - PC6_edited['N_NW_min']

In [65]:
PC6_edited = pd.merge(PC6_edited, PC6[['Postcode','P_NL_ACHTG','P_WE_MIG_A','P_NW_MIG_A']], on = 'Postcode')

In [66]:
PC6_edited['N_NL_add'] = 0
PC6_edited['N_WE_add'] = 0
PC6_edited['N_NW_add'] = 0

PC6_edited['P_NL_ACHTG'] = PC6_edited['P_NL_ACHTG'].mask((PC6_edited['P_NL_ACHTG'] == 0) &
                                                         (PC6_edited['P_WE_MIG_A'] == 0) &
                                                         (PC6_edited['P_NW_MIG_A'] == 0), PC6_edited['P_NL_PC5'])
PC6_edited['P_WE_MIG_A'] = PC6_edited['P_WE_MIG_A'].mask((PC6_edited['P_NL_ACHTG'] == 0) &
                                                         (PC6_edited['P_WE_MIG_A'] == 0) &
                                                         (PC6_edited['P_NW_MIG_A'] == 0), PC6_edited['P_WE_PC5'])
PC6_edited['P_NW_MIG_A'] = PC6_edited['P_NW_MIG_A'].mask((PC6_edited['P_NL_ACHTG'] == 0) &
                                                         (PC6_edited['P_WE_MIG_A'] == 0) &
                                                         (PC6_edited['P_NW_MIG_A'] == 0), PC6_edited['P_NW_PC5'])

PC6_edited['N_NL_add'] = (PC6_edited['undefined_PC6'] * PC6_edited['P_NL_ACHTG'] /100).round()
PC6_edited['N_WE_add'] = (PC6_edited['undefined_PC6'] * PC6_edited['P_WE_MIG_A'] /100).round()
PC6_edited['N_NW_add'] = (PC6_edited['undefined_PC6'] * PC6_edited['P_NW_MIG_A'] /100).round()

PC6_edited['N_NL_add'] = PC6_edited['N_NL_add'].mask(PC6_edited['N_NL_add'] > PC6_edited['N_NL_max'] - PC6_edited['N_NL_min'],
                                                     PC6_edited['N_NL_max'] - PC6_edited['N_NL_min'])
PC6_edited['N_WE_add'] = PC6_edited['N_WE_add'].mask(PC6_edited['N_WE_add'] > PC6_edited['N_WE_max'] - PC6_edited['N_WE_min'],
                                                     PC6_edited['N_WE_max'] - PC6_edited['N_WE_min'])
PC6_edited['N_NW_add'] = PC6_edited['N_NW_add'].mask(PC6_edited['N_NW_add'] > PC6_edited['N_NW_max'] - PC6_edited['N_NW_min'],
                                                     PC6_edited['N_NW_max'] - PC6_edited['N_NW_min'])

PC6_edited['N_NL'] = PC6_edited['N_NL_min'] + PC6_edited['N_NL_add']
PC6_edited['N_WE'] = PC6_edited['N_WE_min'] + PC6_edited['N_WE_add']
PC6_edited['N_NW'] = PC6_edited['N_NW_min'] + PC6_edited['N_NW_add']

PC6_edited['pop_est'] = PC6_edited['N_NL'] + PC6_edited['N_WE'] + PC6_edited['N_NW']

In [67]:
PC6_edited = PC6_edited.drop(columns = ['PC5', 'N_NL_min', 'N_NL_max', 'N_WE_min', 'N_WE_max',
                                        'N_NW_min', 'N_NW_max','P_NL_PC5', 'P_WE_PC5', 'P_NW_PC5',
                                        'undefined_PC6', 'P_NL_ACHTG', 'P_WE_MIG_A','P_NW_MIG_A', 
                                        'N_NL_add', 'N_WE_add', 'N_NW_add'])

In [69]:
PC6_edited.to_csv('../data/processed_data/sociodemographics/sociodemographics.csv', 
                  index = False)